### Creating ``.fit()``, `.compile()` and `.evaluate()` for all the api's.

In this notebook we are going to use subclassing to create our custom `.fit()`, `.compile()` and `.evaluate()` methods that will work on both  of the following api's.

1. Sequantial API
2. Functional API
3. Subclassing API

### Imports

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets
import numpy as np

### Configuring the ``device`` for the environment.


In [2]:
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

### Let's create a model that will train on the `MNIST` dataset.

In [3]:
(X_train, y_train), (X_test, y_test) = datasets.mnist.load_data()
X_train.shape

11493376/11490434 [==============================] - 0s 0us/step


(60000, 28, 28)

In [4]:
def normalize(image):
  image = tf.convert_to_tensor(image.astype('float32'))/255
  return image

In [5]:
X_train_tensors =tf.convert_to_tensor(list(map(normalize, X_train)))
X_test_tensors = tf.convert_to_tensor(list(map(normalize, X_test)))

y_test_tensors = tf.convert_to_tensor(y_test)
y_train_tensors = tf.convert_to_tensor(y_train)

In [6]:
y_test_tensors[:2]

<tf.Tensor: shape=(2,), dtype=uint8, numpy=array([7, 2], dtype=uint8)>

### Custom `evaluate()`, `compile()` and `fit()`:

In [7]:
class Model(keras.Model):
  def __init__(self, model):
    super().__init__(self)
    self.model = model

  # .compile()
  def compile(self, loss, optimizer, metrics):
    self.loss = loss
    self.custom_metrics = metrics
    self. optimizer = optimizer

  # .fit()
  def train_step(self, data):
    x, y = data
    # forward pass
    with tf.GradientTape() as tape:
      y_pred = self.model(x, training=True)
      # loss
      loss = self.loss(y, y_pred)
    # calculate the gradients
    gradients = tape.gradient(loss, self.trainable_variables)
    # update the weights
    self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
    # update the metrics
    self.custom_metrics.update_state(y, y_pred)
    return {"loss": loss, "accuracy": self.custom_metrics.result()}

  def test_step(self, data):
    x, y = data
    y_pred = model(x, training=False)
    loss = self.loss(y, y_pred)
    self.custom_metrics.update_state(y, y_pred)
    return {"loss": loss, "accuracy": self.custom_metrics.result()}


 ### Let's create a `Sequential` model that will use our custom `.fit()`, `.compile()` and `.evaluate()` method.

In [8]:
seq_model = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
], name="seq_model")

seq_model_1 = Model(seq_model)
seq_model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    metrics = keras.metrics.SparseCategoricalAccuracy()           
)
seq_model.fit(X_train_tensors, y_train_tensors, epochs=2, verbose=1, batch_size=32,
          validation_data=(X_test_tensors, y_test_tensors),
          validation_batch_size=16)

Epoch 1/2
1875/1875 [==============================] - 9s 3ms/step - loss: 0.2611 - sparse_categorical_accuracy: 0.9235 - val_loss: 0.1384 - val_sparse_categorical_accuracy: 0.9579
Epoch 2/2
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1157 - sparse_categorical_accuracy: 0.9646 - val_loss: 0.1084 - val_sparse_categorical_accuracy: 0.9651


 ### Let's create a `Functional` model that will use our custom `.fit()`, `.compile()` and `.evaluate()` method.


In [9]:
input_layer = keras.layers.Input(shape=(28, 28, ))
flatten_layer = keras.layers.Flatten()(input_layer)
hidden_1 = keras.layers.Dense(64, activation='relu')(flatten_layer)
hidden_2 = keras.layers.Dense(128, activation="relu")(hidden_1)
output_layer = keras.layers.Dense(10, activation="softmax")(hidden_2)

fn_model = keras.Model(inputs=input_layer, outputs=output_layer, name="fn_model")

fn_model_1 = Model(fn_model)
fn_model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    metrics = keras.metrics.SparseCategoricalAccuracy()           
)
fn_model.fit(X_train_tensors, y_train_tensors, epochs=2, verbose=1, batch_size=32,
          validation_data=(X_test_tensors, y_test_tensors),
          validation_batch_size=16)

Epoch 1/2
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2691 - sparse_categorical_accuracy: 0.9206 - val_loss: 0.1461 - val_sparse_categorical_accuracy: 0.9551
Epoch 2/2
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1176 - sparse_categorical_accuracy: 0.9647 - val_loss: 0.1056 - val_sparse_categorical_accuracy: 0.9686


 ### Let's create a `Subclassing` model that will use our custom `.fit()`, `.compile()` and `.evaluate()` method.

In [34]:
class MNISTModel(keras.Model):
  def __init__(self):
    super().__init__()
    self.flatten_layer = keras.layers.Flatten(input_shape=(28, 28))
    self.dense_1 = keras.layers.Dense(64, activation='relu')
    self.dense_2 = keras.layers.Dense(128, activation='relu')
    self.output_layer = keras.layers.Dense(10, activation='softmax')

  def call(self, x):
    x = self.flatten_layer(x)
    y = self.dense_1(x)
    y = self.dense_2(y)
    y = self.output_layer(y)
    return y
  def model(self):
    x = keras.layers.Input(shape=(28*28,))
    return keras.Model(inputs=[x], outputs=self.call(x))

sub_model_1 = MNISTModel()
sub_model_1.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    metrics = [keras.metrics.SparseCategoricalAccuracy()   ]        
)
sub_model_1.fit(X_train_tensors, y_train_tensors, epochs=2, verbose=1, batch_size=32,
          validation_data=(X_test_tensors, y_test_tensors),
          validation_batch_size=16)

sub_model_1.model().summary()

Epoch 1/2
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2565 - sparse_categorical_accuracy: 0.9247 - val_loss: 0.1385 - val_sparse_categorical_accuracy: 0.9583
Epoch 2/2
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1151 - sparse_categorical_accuracy: 0.9648 - val_loss: 0.1058 - val_sparse_categorical_accuracy: 0.9670
Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
flatten_25 (Flatten)         (None, 784)               0         
_________________________________________________________________
dense_75 (Dense)             (None, 64)                50240     
_________________________________________________________________
dense_76 (Dense)             (None, 128)               8320      
__________________________

### Observations:
The keras subclass api didn't work with the custom `.fit()`, `.train()` and `.evaluate()` but I've learned something about the subclass, which is specifying the input shape.

### Getting an error?
* [Solution](https://stackoverflow.com/questions/67787156/attributeerror-layer-mnist-model-35-has-no-inbound-nodes-tensorflow-keras-subc/67787639#67787639)